In [1]:
# Donchian Weekly Classic Trend Following System
import gta_prices
import numpy as np
import pandas as pd

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'EMG'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
p = [48, 24, 12, 6] # Look back periods.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [3]:
# Function definitions.
def charges(date, commission):
    '''Calculate trading charges.'''
    df = pd.DataFrame(index=date, columns=['date_shift', 'charges'])
    df.date_shift = df.index.values
    df.date_shift = df.date_shift.shift()
    df.charges = np.where(df.index == df.date_shift, 0, commission)
    return df.charges

def donchian(prices, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame()
    df['upr'] = prices.high.rolling(period).max().shift(periods=1)
    df['lwr'] = prices.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df

def shares_div4(shares):
    '''Modify number of shares to be purchased to be divisible by 4.'''
    s = pd.Series(shares.values)
    for i in s.index:
        while s.iloc[i] % 4 != 0:
            s.iloc[i] += 1
    return shares

def state(entry_signal, exit_signal, period):
    '''Calculate trade state signals.'''
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ['entry', 'exit']
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
        and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df.state

def trade_summary(trade_list):
    '''Generate trade summary.'''
    frame = pd.DataFrame(columns=['trade', 'entry', 'volatility','cost', 'exit', 'days', 'profit', 'pct', 'annual'])
    if trade_list.entry.iloc[-1] == 1:
        trade_num = trade_list.index.max()
    else:
        trade_num = trade_list.index.max() + 1
        
    for i in list(range(1, trade_num)):
        df = trade_list.loc[i]
        trade = df.index[0]
        entry = df.date.iloc[0]
        volatility = df.volatility.iloc[0]
        cost = df.cost.iloc[0]
        exit = df.date.iloc[-1]
        days = df.days.iloc[-1]
        profit = df.profit.sum()
        pct = (profit / cost) * 100
        annual = ((1 + pct / 100) ** (365 / days) - 1) * 100
        frame.loc[i] = [trade, entry, volatility, cost, exit, days, profit, pct, annual]
    frame = frame.set_index('trade')
    return frame

pd.set_option("display.max_columns", None)

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm)

In [5]:
# Donchian channels.
for i in range(len(p)):
    globals()[f'p{i + 1}'] = p[i]
    globals()[f'dc{i + 1}'] = donchian(prices, p[i])

In [6]:
# System 1 entries & exits.
s1 = pd.concat([prices, dc1], axis=1)
s1['sys'] = 1
s1['buy'] = np.where(s1.close > s1.upr, 1, 0)
s1['sell'] = np.where(s1.close < s1.mid, 1, 0)
s1['state'] = state(s1.buy, s1.sell, p1)
s1['entry'] = np.where(np.logical_and(s1.state == 1, s1.state.shift(periods=1) == 0), 1, 0)
s1['exit'] = np.where(np.logical_and(s1.state == 0, s1.state.shift(periods=1) == 1), 1, 0)

In [7]:
# System 2 entries & exits.
s2 = pd.concat([prices, dc2], axis=1)
s2['sys'] = 2
s2['buy'] = s1.entry
s2['sell'] = np.where(s2.close < s2.mid, 1, 0)
s2['state'] = state(s2.buy, s2.sell, p2)
s2['entry'] = np.where(np.logical_and(s2.state == 1, s2.state.shift(periods=1) == 0), 1, 0)
s2['exit'] = np.where(np.logical_and(s2.state == 0, s2.state.shift(periods=1) == 1), 1, 0)

In [8]:
# System 3 entries & exits.
s3 = pd.concat([prices, dc3], axis=1)
s3['sys'] = 3
s3['buy'] = s1.entry
s3['sell'] = np.where(s3.close < s3.mid, 1, 0)
s3['state'] = state(s3.buy, s3.sell, p3)
s3['entry'] = np.where(np.logical_and(s3.state == 1, s3.state.shift(periods=1) == 0), 1, 0)
s3['exit'] = np.where(np.logical_and(s3.state == 0, s3.state.shift(periods=1) == 1), 1, 0)

In [9]:
# System 4 entries & exits.
s4 = pd.concat([prices, dc4], axis=1)
s4['sys'] = 4
s4['buy'] = s1.entry
s4['sell'] = np.where(s4.close < s4.mid, 1, 0)
s4['state'] = state(s4.buy, s4.sell, p4)
s4['entry'] = np.where(np.logical_and(s4.state == 1, s4.state.shift(periods=1) == 0), 1, 0)
s4['exit'] = np.where(np.logical_and(s4.state == 0, s4.state.shift(periods=1) == 1), 1, 0)

In [10]:
# Trade list indexed by date.
td = pd.concat([s1[s1.entry == 1] , s1[s1.exit == 1], s2[s2.exit == 1], s3[s3.exit == 1], s4[s4.exit == 1]], axis=0)
td = td.sort_index()

In [11]:
# Position size (buy).
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close), 0)
td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility), 0)
td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)
td.shares = shares_div4(td.shares) # Modify number of shares to be purchased to be divisible by 4.
td.risk_amt = np.where(td.entry == 1, (td.close * td.shares), 0) # Adjust risk amount for revised share count.

In [12]:
# Position size (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        shares = row['shares']
    elif row['exit'] == 1:
        td.at[index, 'shares'] = int(shares / 4)

In [13]:
# Charges.
td['charges'] = charges(td.index, commission)

In [14]:
# Stamp duty.
td['sduty'] = np.where(td.entry==1, ((sduty / 100) * td.close * td.shares), 0)

In [15]:
# Cost (buy).
td['cost'] = np.where(td.entry == 1, ((td.close * td.shares) + td.charges + td.sduty), 0)

In [16]:
# Cost (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        cost = row['cost']
    elif row['exit'] == 1:
        td.at[index, 'cost'] = (cost / 4)

In [17]:
# Value (sell).
td['value'] = np.where(td.exit == 1, ((td.close * td.shares) - td.charges), 0)

In [18]:
# Profit.
td['profit'] = np.where(td.exit == 1, td.value - td.cost, 0)

In [19]:
# Cumulative profit.
td['profit_cum'] = td.profit.cumsum()

In [20]:
# Percentage return.
td['pct'] = np.where(td.exit == 1, ((td.profit / td.cost) * 100), 0)

In [21]:
# Trade duration.
td['days'] = 0
for index, row in td.iterrows():
    if row['entry'] == 1:
        start_date = index
    elif row['exit'] == 1:
        td.at[index, 'days'] = index - start_date
td.days = td.days.astype('timedelta64[D]')
td.days = td.days.dt.days

In [22]:
# Annual percentage return.
td['annual'] = ((np.power(1 + td.profit / td.cost, (365 / td.days)) - 1) * 100)

In [23]:
# Trade list indexed by trade.
td['trade'] = td.state.cumsum()
td = td.reset_index()
td = td.set_index('trade')
td.round(2)

,date,open,high,low,close,upr,lwr,mid,sys,buy,sell,state,entry,exit,volatility,risk_amt,shares,charges,sduty,cost,value,profit,profit_cum,pct,days,annual
trade,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2001-05-18,1.69,1.77,1.68,1.77,1.74,0.91,1.32,1,1,0,1,1,0,0.25,5967.42,3376,11.95,29.84,6009.20,0.00,0.00,0.00,0.00,0,0.00
1,2001-07-13,1.87,1.87,1.71,1.72,1.91,1.69,1.80,4,0,1,0,0,1,0.00,0.00,844,11.95,0.00,1502.30,1441.33,-60.97,-60.97,-4.06,56,-23.66
1,2001-07-13,1.87,1.87,1.71,1.72,1.91,1.56,1.74,3,0,1,0,0,1,0.00,0.00,844,0.00,0.00,1502.30,1453.28,-49.02,-109.99,-3.26,56,-19.44
1,2001-07-20,1.72,1.72,1.55,1.60,1.91,1.36,1.64,2,0,1,0,0,1,0.00,0.00,844,11.95,0.00,1502.30,1338.45,-163.85,-273.84,-10.91,63,-48.78
1,2002-02-22,2.25,2.25,1.83,1.90,2.66,1.38,2.02,1,0,1,0,0,1,0.00,0.00,844,11.95,0.00,1502.30,1595.70,93.40,-180.44,6.22,280,8.18
2,2003-05-16,2.04,2.24,2.03,2.23,2.16,1.53,1.84,1,1,0,1,1,0,0.17,8700.45,3904,11.95,43.50,8755.91,0.00,0.00,-180.44,0.00,0,0.00
2,2003-07-25,2.42,2.43,2.36,2.38,2.51,2.27,2.39,4,0,1,0,0,1,0.00,0.00,976,11.95,0.00,2188.98,2308.10,119.12,-61.31,5.44,70,31.83
2,2003-08-22,2.33,2.41,2.28,2.34,2.51,2.22,2.36,3,0,1,0,0,1,0.00,0.00,976,11.95,0.00,2188.98,2272.87,83.89,22.58,3.83,98,15.04
2,2004-05-14,3.14,3.17,3.02,3.10,3.54,2.68,3.11,2,0,1,0,0,1,0.00,0.00,976,11.95,0.00,2188.98,3010.82,821.84,844.42,37.54,364,37.67


In [24]:
# Trade summary.
tds = trade_summary(td)
tds.round({'volatility': 3, 'pct': 1, 'annual': 1})

,entry,volatility,cost,exit,days,profit,pct,annual
trade,,,,,,,,
1,2001-05-18,0.251,6009.204688,2002-02-22,280,-180.436288,-3.0,-3.9
2,2003-05-16,0.172,8755.906672,2004-06-25,406,1614.312528,18.4,16.4
3,2005-07-08,0.141,10705.601848,2007-08-10,763,3656.471352,34.2,15.1
4,2009-10-16,0.293,5157.156040,2010-01-29,105,-986.074040,-19.1,-52.2
5,2011-01-07,0.186,8142.416080,2011-03-18,70,-1330.962080,-16.3,-60.6
6,2013-05-03,0.265,5699.446000,2013-06-07,35,-1004.988000,-17.6,-86.8
7,2014-07-04,0.180,8385.630100,2015-06-19,350,1953.919900,23.3,24.4
8,2017-04-28,0.170,8872.263160,2018-02-23,301,687.736840,7.8,9.5
9,2021-04-09,0.182,8315.018800,2022-01-28,294,1748.477200,21.0,26.7


In [25]:
cost_total = tds.cost.sum()
cost_total

70042.64338800001

In [26]:
profit_total = tds.profit.sum()
profit_total

6158.457412

In [27]:
pct_total = (profit_total / cost_total) * 100
pct_total

8.792440025264797

In [28]:
days_total = (tds.exit.iloc[-1] - tds.entry.iloc[0])
days_total = days_total.days
days_total

7560

In [29]:
annual_total = ((1 + pct_total / 100) ** (365 / days_total) - 1) * 100
annual_total

0.4076959742262076